This notebook visualizes the correspondence between multiple high accuracy fast sampling streams, on two different platforms, while the phones were moving.

Remember to load the `stationary`, `high+1sec` and `loitering`, `high+1sec` data from `Pull_entries_from_server` if you have not already done so.

In [ ]:
import sys
import logging
from uuid import UUID

import emission.core.get_database as edb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as pydt
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt
import numpy as np

logging.getLogger().setLevel(logging.DEBUG)
logging.debug(sys.path)

iphone_ids = [UUID("079e0f1a-c440-3d7c-b0e7-de160f748e35"), UUID("c76a0487-7e5a-3b17-a449-47be666b36f6"), 
              UUID("c528bcd2-a88b-3e82-be62-ef4f2396967a"), UUID("95e70727-a04e-3e33-b7fe-34ab19194f8b")]
android_ids = [UUID("e471711e-bd14-3dbe-80b6-9c7d92ecc296"), UUID("fd7b4c2e-2c8b-3bfa-94f0-d1e3ecbd5fb7"),
               UUID("86842c35-da28-32ed-a90e-2da6663c5c73"), UUID("3bc0f91f-7660-34a2-b005-5c399598a369")]

In [ ]:
iphone_ts = map(lambda u: esta.TimeSeries.get_time_series(u), iphone_ids)
android_ts = map(lambda u: esta.TimeSeries.get_time_series(u), android_ids)

In [ ]:
ranges_df = pd.read_csv("../results_spring_2016/ranges.csv", error_bad_lines=False)

In [ ]:
ranges_df.head()

In [ ]:
regime_grouped_ranges_df = ranges_df.groupby(['state', 'android_regime', 'ios_regime'])

In [ ]:
invalid_df = pd.read_csv("../results_spring_2016/invalid.csv")

In [ ]:
invalid_df

In [ ]:
import emission.analysis.point_features as pf
import attrdict as ad

In [ ]:
def calDistance(row):
    return pf.calDistance(ad.AttrDict({"latitude": row[0], "longitude": row[1]}),
                          ad.AttrDict({"latitude": row[3], "longitude": row[4]}))

In [ ]:
import scipy.interpolate as spi
import numpy as np

In [ ]:
np.array([[1]]).shape

In [ ]:
def get_interp_distance_df(android_ts, ios_ts, start_ts, end_ts):
    android_df = android_ts.get_data_df("background/location", estt.TimeQuery('metadata.write_ts', start_ts, end_ts))
    ios_df = ios_ts.get_data_df("background/location", estt.TimeQuery('metadata.write_ts', start_ts, end_ts))
    android_latf = spi.interp1d(x=android_df.ts, y=android_df.latitude, bounds_error=False,
                                fill_value="extrapolate")
    android_lngf = spi.interp1d(x=android_df.ts, y=android_df.longitude, bounds_error=False, 
                                fill_value="extrapolate")
    ios_latf = spi.interp1d(x=ios_df.ts, y=ios_df.latitude, bounds_error=False, fill_value=(ios_df.latitude.iloc[0], ios_df.latitude.iloc[-1]))
    ios_lngf = spi.interp1d(x=ios_df.ts, y=ios_df.longitude, bounds_error=False, fill_value=(ios_df.longitude.iloc[0], ios_df.longitude.iloc[-1]))
    ts_new = np.arange(int(start_ts), int(end_ts), 1)
    fmt_time_new = map(lambda ts: pydt.datetime.fromtimestamp(ts).isoformat(), ts_new)
    interp_df = pd.DataFrame({"ts": ts_new, "fmt_time": fmt_time_new,
                            "android_lat": android_latf(ts_new), "android_lng": android_lngf(ts_new),
                            "ios_lat": ios_latf(ts_new), "ios_lng": ios_lngf(ts_new)}, index=ts_new)
    same_point_distances = np.abs(interp_df.apply(calDistance, axis=1))
    interp_df["same_point_distances"] = same_point_distances
    return interp_df

In [ ]:
def get_points_from_ts_for_range(ts, start_ts, end_ts):
    return ts.get_data_df("background/location", estt.TimeQuery('metadata.write_ts', start_ts, end_ts))

In [ ]:
def get_points_for_range(ts, range_df):
    points_range_df = map(lambda row: get_points_from_ts_for_range(ts, row['start_ts'], row['end_ts']), range_df.to_dict('records'))
    return points_range_df

In [ ]:
def get_points_for_group(ts_list, range_df):
    return map(lambda ts: get_points_for_range(ts, range_df), ts_list)

In [ ]:
import emission.analysis.plotting.leaflet_osm.our_plotter as lo
import emission.analysis.plotting.geojson.geojson_feature_converter as gfc
import emission.analysis.plotting.leaflet_osm.ipython_helper as ipy

In [ ]:
iphone_high_fast_loitering_df = pd.DataFrame()
for key, df in regime_grouped_ranges_df:
    if key[0] == 'loitering' and key[1] == 'high+1sec':
        # one row for each phone
        # each row contains len(stationary ranges) dataframes
        iphone_range_location_df_list = get_points_for_group(iphone_ts, df)

In [ ]:
android_high_fast_loitering_df = pd.DataFrame()
for key, df in regime_grouped_ranges_df:
    if key[0] == 'loitering' and key[1] == 'high+1sec':
        # one row for each phone
        # each row contains len(stationary ranges) dataframes
        android_phone_range_location_df_list = get_points_for_group(android_ts, df)

In [ ]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(iphone_range_location_df_list[0][1])]) + 
                 lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(android_phone_range_location_df_list[0][1].sample(frac=0.1))])])

In [ ]:
time_df = android_phone_range_location_df_list[0][1]
interp_df = get_interp_distance_df(android_ts[0], iphone_ts[0], time_df.iloc[0].ts, time_df.iloc[-1].ts)

In [ ]:
interp_df.same_point_distances.plot(kind='hist', bins = 100)

In [ ]:
ax = interp_df[interp_df.same_point_distances < 100].same_point_distances.plot(kind='hist', bins = 100, figsize=(12,4))
tick_list = ax.set_xticks(np.arange(0,100,2))

In [ ]:
ax = interp_df[interp_df.same_point_distances < 100].same_point_distances.hist(bins=100, figsize=(15,3), cumulative=True)
tick_list = ax.set_xticks(np.arange(0,100,2))

In [ ]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(interp_df.sample(frac=0.01), ts="ts", latitude="ios_lat", longitude="ios_lng")]) +
lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(interp_df.sample(frac=0.01), ts="ts", latitude="android_lat", longitude="android_lng")])])

In [ ]:
cleaned_stationary_df_18 = interp_df[interp_df.same_point_distances <= 18]

In [ ]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(cleaned_stationary_df_18.sample(frac=0.01), ts="ts", latitude="ios_lat", longitude="ios_lng")]) +
lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(cleaned_stationary_df_18.sample(frac=0.01), ts="ts", latitude="android_lat", longitude="android_lng")])])

In [ ]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(cleaned_stationary_df_18.sample(frac=0.01), ts="ts", latitude="ios_lat", longitude="ios_lng")]) +
lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(cleaned_stationary_df_18.sample(frac=0.01), ts="ts", latitude="android_lat", longitude="android_lng")])])

In [ ]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(cleaned_stationary_df_18.sample(frac=0.01), ts="ts", latitude="ios_lat", longitude="ios_lng")]) +
lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(cleaned_stationary_df_18.sample(frac=0.01), ts="ts", latitude="android_lat", longitude="android_lng")])])